# Capstone project

This notebook will be mainly used for the capstone project.

## Week 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Week 3: Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

In [3]:
import requests
base_url_wikipedia = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_wikipedia = requests.get(base_url_wikipedia).content

### Converting the table headings into list of column names

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_wikipedia, 'html.parser')
table = soup.find('table', class_='wikitable')
headings = table.find_all('th')

column_names = []
for heading in headings:
    column_names.append(heading.text.strip())
    
print(len(column_names))

3


### Converting the table cells to a list of postal codes

In [5]:
table_body = table.find('tbody')
rows = table_body.find_all('tr')

postal_codes = []

for row in rows:
    postal_code = []
    cells = row.find_all('td')
    
    for cell in cells:
        postal_code.append(cell.text.strip())
        
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if len(postal_code) < 3 or postal_code[1] == 'Not assigned':
        continue
        
    # If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    if postal_code[2] == '':
        postal_code[2] = postal_code[1]
        
    # Change slash separated neighborhoods into comma separated ones
    if '/' in postal_code[2]:        
        postal_code[2] = postal_code[2].replace(' /', ',')
    
    postal_codes.append(postal_code)
    
# print(postal_codes[0:10])

### Converting lists into a pandas dataframe

In [6]:
df = pd.DataFrame(postal_codes, columns=column_names) 
df.shape

(103, 3)

### Changing postal codes into latitude and lognitude values

In [7]:
"""!pip install geocoder"""

'!pip install geocoder'

In [8]:
"""import geocoder # import geocoder

for pc in postal_codes:
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto'.format(pc[0]))
        lat_lng_coords = g.latlng
        
    # saving the coordinates into postal_codes list
    postal_codes[3] = lat_lng_coords[0]
    postal_codes[4] = lat_lng_coords[1]
    
column_names.extend(['latitude', 'lognitude'])"""

"import geocoder # import geocoder\n\nfor pc in postal_codes:\n    lat_lng_coords = None\n\n    # loop until you get the coordinates\n    while(lat_lng_coords is None):\n        g = geocoder.google('{}, Toronto'.format(pc[0]))\n        lat_lng_coords = g.latlng\n        \n    # saving the coordinates into postal_codes list\n    postal_codes[3] = lat_lng_coords[0]\n    postal_codes[4] = lat_lng_coords[1]\n    \ncolumn_names.extend(['latitude', 'lognitude'])"

#### Since the api didnt work at all, let's download the csv file

In [9]:
df_from_api = pd.read_csv('https://cocl.us/Geospatial_data')
df_from_api.set_index('Postal Code', inplace=True)
df_from_api.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [10]:
for pc in postal_codes:
    row_data = df_from_api.loc[ pc[0] , : ]
    pc.extend([row_data['Latitude'], row_data['Longitude']])
# print(postal_codes)

In [11]:
column_names.extend(['latitude', 'lognitude'])

In [12]:
df_new = pd.DataFrame(postal_codes, columns=column_names)
df_new.head()
# df_new.shape

,Postal code,Borough,Neighborhood,latitude,lognitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
